In [12]:
import os
from collections import Counter
import numpy as np
import random
import tables
import requests
import sqlite3
import sevendigital_preview 
from urllib2 import HTTPError
random.seed(1234)

In [13]:
#reload(sevendigital_preview)
oauthkey = ['7daju79vrfv8','7dvmrhbyzu2j','7de2vxyvdka7'][1]
secret = ['yx7bh8eje83537mm','zphtpamzhgz49vj6','fctd2pwvzqceautf'][1]
sevendigital_preview.set_credentials(oauthkey, secret)

In [14]:
#load metadata
#build cluster dict from db
conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
c = conn.cursor()
c.execute("SELECT msd_id, audio_path FROM tracks WHERE audio_path IS NOT NULL ORDER BY msd_id")
tracks = [el[0] for el in c.fetchall()]
conn.close()



In [15]:
# new approach starting from likes
msd_known_jams = set() #this is a set of jam ids which we can map to msd ids
with open('msd_data/jam_to_msd.tsv') as f:
    for line in f:
        msd_known_jams.add(line.split('\t')[0].strip())

In [18]:
#load all likes into memory
likes = []
lines = 0
with open('msd_data/jam_dump/archive/likes.tsv') as f:
    next(f)
    for line in f:
        lines += 1
        like = {'user': line.split('\t')[0], 'jam': line.split('\t')[1].strip()}
        #intersect likes with msd_known_jams
        try:
            if get_msd_id[like['jam']] in tracks:
                like['jam'] = get_msd_id[like['jam']]
                likes.append(like)
        except KeyError:
            pass

In [25]:
lines

5968624

In [30]:
#count jams by user and compute average and variance
user_jam_counter = Counter([x['user'] for x in likes])
np.mean(user_jam_counter.values()), np.std(user_jam_counter.values())

(39.183291592128803, 306.91883726058239)

In [49]:
len(likes)

1095173

In [19]:
#now select users that have between 20 and 200 jams
#' THIS INTRODUCES BIAS IN THE SUBSET'
user_subset = set()
for user in user_jam_counter.keys():
    if 20 <= user_jam_counter[user] <= 200:
        user_subset.add(user)

NameError: name 'user_jam_counter' is not defined

In [ ]:
#randomly choose 200 users to further restrict user_subset
user_sample = set(random.sample(user_subset,250))

In [ ]:
#create a like subset
like_subset = []
for like in likes:
        if like['user'] in user_sample:
            like_subset.append(like)

In [ ]:
len(like_subset)

In [32]:
#count how many unique jams we get
jam_counter = Counter([x['jam'] for x in likes])
len(jam_counter.keys())

8003

In [17]:
def jam_mapping(path):
    mapping = {}
    with open(path) as f:
        for line in f:
            values = line.split('\t')
            mapping[values[0].strip('\n')] = values[1].strip('\n')
    return mapping
get_msd_id = jam_mapping('msd_data/jam_to_msd.tsv')
get_jam_id = inv_map = {v: k for k, v in get_msd_id.iteritems()}

In [11]:
#set of sampled jams as msd_ids
sampled_jams = set()
for jam_id in jam_counter.keys():
    sampled_jams.add(get_msd_id[jam_id])
len(sampled_jams)

8450

In [12]:
#create dictionary that maps msd_id to song metadata
f = tables.open_file('msd_data/msd_summary_file.h5', mode='r')
tracks = {}
for idx, msd_id in enumerate(f.root.analysis.songs.cols.track_id):
    if msd_id in sampled_jams:
        tracks[msd_id] = {'track_7digitalid':f.root.metadata.songs.cols.track_7digitalid[idx],
                          'release_7digitalid':f.root.metadata.songs.cols.release_7digitalid[idx], 
                          'artist_7digitalid':f.root.metadata.songs.cols.artist_7digitalid[idx],
                          'release': f.root.metadata.songs.cols.release[idx],
                          'title': f.root.metadata.songs.cols.title[idx], 
                          'artist': f.root.metadata.songs.cols.artist_name[idx]}
f.close()

In [ ]:
def create_db():
    conn = sqlite3.connect("msd_data/analysis_data/sampled_likes_tracks.db")
    c = conn.cursor()
    c.execute('''CREATE TABLE tracks (msd_id text, track_7digitalid integer, som_cluster integer, igmn_cluster integer, audio_path text)''')
    conn.commit()
    conn.close()
#create_db()

In [13]:
os.chdir('/home/kayibal/sc-recom/code/msd_data/analysis_data')
no_audio_set = set()
with open('no_audio.txt') as f:
    for line in f:
        no_audio_set.add(line.strip())

In [ ]:
no_audio = []

#database
conn = sqlite3.connect("msd_data/analysis_data/sampled_likes_tracks.db")
c = conn.cursor()

#iterate through selected tracks
os.chdir('/home/kayibal/sc-recom/code/msd_data/analysis_data')
it = 1
for msd_id, track in tracks.iteritems():
    print "%d : %s \t %s - %s" % (it, msd_id, track['title'], track['artist'])
    it += 1
    dir_path = "%s/%s/%s" % (msd_id[2],msd_id[3], msd_id[4])
    audio_path = os.path.join(dir_path, msd_id + ".mp3")
    if not (os.path.exists(audio_path)) and not msd_id in no_audio_set :
        try:
            audio_url, track_7digitalid = sevendigital_preview.get_url_from_dict(track)
        except HTTPError:
            print "HTTP Error"
        if audio_url != None:
            r = requests.get(audio_url, headers={'Connection':'close'})
            if r.status_code == 404: 
                #sometimes the 7digital ids are old and not used anymore so we use other methods to get the track
                print "\t retrying request"
                try:
                    audio_url, track_7digitalid = sevendigital_preview.get_url_from_dict(track, deprecated_id = True)
                except HTTPError:
                    print "HTTP Error"
                if audio_url != None:
                        r = requests.get(audio_url, headers={'Connection':'close'})
            if r.ok:
                #write audio file
                if not os.path.exists(dir_path):
                    os.makedirs(dir_path)
                with open(audio_path, 'wb') as handle:
                    for block in r.iter_content(8*1024):
                        handle.write(block)
                #create db_entry
                query = "INSERT INTO tracks (msd_id, track_7digitalid, audio_path) VALUES ('%s',%d,'%s')"
                query = query % (msd_id, track_7digitalid, audio_path)
                c.execute(query)
                conn.commit()
                print "\tSUCCESS AUDIO DOWNLOADED"
            else:
                print "\t no audio: Request failed with code: %d \n request url: %s" % (r.status_code, audio_url)
                no_audio.append(msd_id)
                if "limit" in r.text.lower() or r.status_code == 401:
                    print "\tLIMIT REACHED"
                    break
                continue
            r.connection.close()
        else:
            print "\t no audio: URL lookup failed"
            no_audio.append(msd_id)
            continue
    else:
        print "\tskipped file already exists"
conn.close()
'''
with open("no_audio.txt", "w") as f:
    for item in no_audio:
        f.write("%s\n" % item)
'''
os.chdir('/home/kayibal/sc-recom/code')


In [47]:
os.chdir('/home/kayibal/sc-recom/code')
reload(sevendigital_preview)

<module 'sevendigital_preview' from 'sevendigital_preview.pyc'>

In [ ]:
hopeless=[]

#database
conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
c = conn.cursor()

#iterate through selected tracks
os.chdir('/home/kayibal/sc-recom/code/msd_data/analysis_data')
it = 1

for msd_id in no_audio_set:
    track = tracks[msd_id]
    print "%d : %s \t %s - %s" % (it, msd_id, track['title'], track['artist'])
    it += 1
    dir_path = "%s/%s/%s" % (msd_id[2],msd_id[3], msd_id[4])
    audio_path = os.path.join(dir_path, msd_id + ".mp3")
    if not (os.path.exists(audio_path)):
        audio_url = None
        try:
            audio_url, track_7digitalid = sevendigital_preview.get_url_simple(track)
        except HTTPError,e:
            break
        if audio_url != None:
            r = requests.get(audio_url, headers={'Connection':'close'})
            if r.ok:
                #write audio file
                if not os.path.exists(dir_path):
                    os.makedirs(dir_path)
                with open(audio_path, 'wb') as handle:
                    for block in r.iter_content(8*1024):
                        handle.write(block)
                #create db_entry
                query = "INSERT INTO tracks (msd_id, track_7digitalid, audio_path) VALUES ('%s',%d,'%s')"
                query = query % (msd_id, track_7digitalid, audio_path)
                c.execute(query)
                conn.commit()
                print "\tSUCCESS AUDIO DOWNLOADED"
            else:
                print "\t no audio: Request failed with code: %d \n request url: %s" % (r.status_code, audio_url)
                hopeless.append(msd_id)
                if "limit" in r.text.lower() or r.status_code == 401:
                    print "\tLIMIT REACHED"
                    break
                continue
            r.connection.close()
        else:
            print "\t no audio: URL lookup failed"
            hopeless.append(msd_id)
            continue
    else:
        print "\tskipped file already exists"
conn.close()

In [ ]:
#database
conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
c = conn.cursor()
c.execute("SELECT msd_id FROM tracks WHERE audio_path IS NOT NULL ORDER BY msd_id")
tracks = c.fetchall()
conn.close()

tracks = [t[0] for t in tracks]
tracks_set = set(tracks)

In [ ]:
#update like subset to exclude hopeless jams
#analysis_failed = ['TRLHQBM128F92D31BD', 'TRRJMYM128F4259501', 'TRRVWMI128F92E7D38','TRTOACI128F42819C7', 'TRBHQRU128F93129F9', 'TRIYPSF12903CDFAB8', 'TRNNKZA128F4226422']
#hopeless_set = set(hopeless + analysis_failed) #set of msd_ids
temp = []

for like in like_subset:
    if get_msd_id[like['jam']] in tracks_set:
        temp.append(like)
len(temp)

In [ ]:
like_subset_old = like_subset
like_subset = temp

In [56]:
from math import floor

user_likes = {}
splitted_likes = {}
new_likes = {}
p = 0.9
for like in likes:
    if user_likes.has_key(like['user']):
        user_likes[like['user']].append(like['jam'])
    else:
        user_likes[like['user']] = [like['jam']]

for key,values in user_likes.iteritems():
    if len(user_likes[key]) > 10:
        new_likes[key] = Counter(values)
    
for user in new_likes.keys():
    lks = new_likes[user].keys()
    first = int(floor(len(lks) * p))
    last = len(lks) - first
    random.shuffle(lks)
    splitted_likes[user] = {'rec':lks[:first], 'eval':lks[-last:]}

In [57]:
len(splitted_likes)

5370

In [52]:
import pickle
os.chdir('/home/kayibal/sc-recom/code/msd_data/')
with open( "eval_dic_big.p", "wb" ) as f:
    pickle.dump(splitted_likes, f)
with open( "like_dic_big.p", "wb" ) as f:
    pickle.dump(user_likes, f)

In [11]:
len(likes)

NameError: name 'likes' is not defined

In [ ]:
#calcuate usage prediction
for user in user_likes.keys():
    recommended_tracks[user] = get_recommendations(splitted_likes[user]['rec'])
    tp += len(set(splitted_likes[user]['rec']) & set(recommended_tracks[user]))
    fp += len(set(recommended_tracks[user]) - set(splitted_likes[user]['rec']))
    fn += len(set(splitted_likes[user]['rec']) - set(recommended_tracks[user]))
    tn += all_tracks - len(set(splitted_likes[user]['rec']) | set(recommended_tracks[user]))

<h2> About Coverage </h2>
<p>
Coverage will be 100% as recommendations are selected using clustering and each song belongs to a cluster. This imposes a very high amount of recommended tracks using the SOM approach but might be much lower using IGMN with a high senibilty paramater.
</p>

In [ ]:
from sklearn.cross_validation import KFold
user_4fold
for key in user_likes.keys():
    

In [58]:
m = np.arange(25).reshape((5,5))

In [61]:
idx = np.where(np.diagonal(m)<18)

In [74]:
diag = np.diag(m).copy()
idx = np.where(diag<18)
diag[idx] = 100

In [76]:
np.fill_diagonal(m,diag)

In [77]:
m

array([[100,   1,   2,   3,   4],
       [  5, 100,   7,   8,   9],
       [ 10,  11, 100,  13,  14],
       [ 15,  16,  17,  18,  19],
       [ 20,  21,  22,  23,  24]])